In [7]:
# Imports
import PySimpleGUI as sg
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os
import scipy
import numpy as np

In [128]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
file_path = os.path.join(parent_directory, "model", "monkey_data.mat")

# Load the .mat file
mat_data = scipy.io.loadmat(file_path)
inp_data = mat_data['inp']
print(inp_data.shape)
for ind,trash in enumerate(inp_data[0]):
    inp_data[0][ind] = inp_data[0][ind][20]
hold_data = inp_data

mvm_data = mat_data['targ']
print(mvm_data.shape)
print(mvm_data[0][0].shape)
print(mvm_data[0][1:200])

(1, 502)
(1, 502)
(50, 298)
[array([[ 0.        ,  0.        ,  0.        , ...,  0.00745155,
          0.01288072,  0.01680898],
        [ 0.        ,  0.        ,  0.        , ...,  0.08523967,
          0.06180861,  0.04126479],
        [ 0.        ,  0.        ,  0.        , ...,  0.00956578,
          0.00586271,  0.00302448],
        ...,
        [ 0.        ,  0.        ,  0.        , ..., -0.37760645,
         -0.34920594, -0.32296256],
        [ 0.        ,  0.        ,  0.        , ..., -0.17194301,
         -0.17086133, -0.16869355],
        [ 0.        ,  0.        ,  0.        , ..., -0.20003636,
         -0.19653433, -0.1907099 ]])
 array([[ 0.        ,  0.        ,  0.        , ...,  0.01512346,
          0.01981549,  0.0228777 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.08172533,
          0.06504852,  0.05032363],
        [ 0.        ,  0.        ,  0.        , ...,  0.00899092,
          0.0049552 ,  0.00193927],
        ...,
        [ 0.        ,  0.  

In [119]:
# Akinesia : want some sort of metric for how long movement initialization is delayed
# Returns the average delay over trial per muscle from 'test' data compared to 'base' data, 
#       using the hold signals as specified in input
#       Also returns some measure of its statistical significance
#
#       input is shape [1,numtrials][1][length of each trial]
#       test and base should have shape [1,numtrials][50][length of each trial]

def akinesia(test, base, input):
    trials = input[0]
    num_trials = input.shape[1]
    all_trial_diffs = np.zeros((num_trials, 50))
    for ind,trial in enumerate(trials):
        inxs = np.argwhere(trial<1)
        go_cue = inxs[0]
        
        # 50 x 200 smth trial
        curr_base_trial = base[0][ind]
        muscle_mvmts_base = np.zeros((50))
        for i, muscle in enumerate(curr_base_trial):
            mvms = np.argwhere(muscle!=0)
            if mvms.size != 0:
                muscle_mvmts_base[i] = mvms[0]
                #print('first mvmt for muscle ' + str(i) + ' at time ' + str(mvms[0]))
        curr_test_trial = test[0][ind]
        muscle_mvmts_test = np.zeros((50))
        for i, muscle in enumerate(curr_test_trial):
            mvms = np.argwhere(muscle!=0)
            if mvms.size != 0:
                muscle_mvmts_test[i] = mvms[0]
                #print('first mvmt for muscle ' + str(i) + ' at time ' + str(mvms[0]))

        all_trial_diffs[ind] = muscle_mvmts_test - muscle_mvmts_base
    
    avg_delay = np.average(all_trial_diffs, axis = 0)

    # missing statistical test
    return avg_delay

        


In [120]:
akinesia(mvm_data, mvm_data, hold_data)

/var/folders/q8/wlhng4j113sg8_tllhtdj7_m0000gn/T/ipykernel_45684/2781912547.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  muscle_mvmts_base[i] = mvms[0]
/var/folders/q8/wlhng4j113sg8_tllhtdj7_m0000gn/T/ipykernel_45684/2781912547.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  muscle_mvmts_test[i] = mvms[0]


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [134]:
# Hypokinesia : movement amplitude, take the integral under the velocity for ‘distance traveled’
#       comparable to the ‘pre-parkinson’s’ state
#       want to have one muscle to the same muscle comparison
#       maybe take a weighted average according to muscle size?

def hypokinesia(test, base):
    trials_base = base
    num_trials_base = base.shape[0]
    all_trial_int_base = np.zeros((num_trials_base, 50))
    
    for ind,trial in enumerate(trials_base):
        all_trial_int_base[ind] = np.sum(trial, axis = 1)
    
    trials_test = test
    num_trials_test = test.shape[0]
    all_trial_int_test = np.zeros((num_trials_test, 50))
    for ind,trial in enumerate(trials_test):
        all_trial_int_test[ind] = np.sum(trial, axis = 1)
    
    avg_dist_base = np.average(all_trial_int_base, axis = 0)
    avg_dist_test = np.average(all_trial_int_test, axis = 0)

    mean_dist_base = np.sum(avg_dist_base)
    print(mean_dist_base)
    std_dist_base = scipy.stats.tstd(np.sum(all_trial_int_base, axis = 1))
    print(std_dist_base)
    mean_dist_test = np.sum(avg_dist_test)
    std_dist_test = scipy.stats.tstd(np.sum(all_trial_int_test, axis = 1))

    t_score = (mean_dist_base - mean_dist_test) / np.sqrt((std_dist_base**2 / num_trials_base) + (std_dist_test**2 / num_trials_test))
    df = num_trials_base + num_trials_test - 2
    p_value = 2 * (1 - scipy.stats.t.cdf(np.abs(t_score), df))
    return avg_dist_test - avg_dist_base, p_value
    


In [135]:
hypokinesia(mvm_data[0][1:200],mvm_data[0][201:])

24.252956182381162
21.082723542811934


(array([ -0.88682042,  -1.03370489,   0.51135061,  -1.03733414,
          0.3887673 ,  -0.5214623 ,   0.83852205,   0.78392459,
         -0.12628594,   0.06644752,   0.26988524,   0.48259813,
          0.57073041,   0.76526664,  -0.08484518,   1.03418699,
          0.43321781,   0.43321785,   0.43457038,   7.83311722,
          7.99628398,   8.84094954,  -0.36903877,  -0.12539664,
          3.09317558,   8.38183711,   6.35938441,  -4.56074824,
         -3.92447546,  -5.00927421,  -6.10983353,  -7.89819015,
         -5.63177134,  -5.82582487,  -9.76983551, -10.94760671,
        -11.91864718,  -8.55059387,  -8.78837676,  -6.50142189,
         10.54820918,  10.42680171,   8.31272167,  10.10693028,
          6.94285575,   8.45970193,  10.13571431,   1.09868018,
         -7.34650541,  -4.26709092]),
 0.0025099941612560794)

In [138]:
print(mvm_data[0][200:].shape)

(302,)
